In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset
from copy import deepcopy
from transformers import (AutoTokenizer,
                         BertForSequenceClassification,
                         Trainer,
                         TrainingArguments,
                         TrainerCallback,
                         get_linear_schedule_with_warmup
                          )
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
# import wandb
# wandb.login()
# wandb.init(project="synthetic_tweets")
import ast
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import os
print(os.getpid())

3965


In [ ]:

df['tweet_no_url'] = df['tweet_no_url'].apply(ast.literal_eval) # tweet_no_url
df['tweets_concatenated'] = df['tweet_no_url'].apply(lambda x: ' '.join([item for item in x if item]) if isinstance(x, list) else '')

# df_synthetic = pd.read_json("/mntssd/mnt3/shanshanbai/nlpinearthobservation/synthetic_data/6000/merged_data.jsonl", lines=True)
# df_synthetic['tweets_concatenated'] = df['tweets'].apply(lambda x: ' '.join([item for item in x if item]) if isinstance(x, list) else '')
# df_synthetic['mapped_class'] = ['residential'] * 3000 + ['commercial'] * (len(df) - 3000)


NameError: name 'df' is not defined

In [ ]:
# Step 1: Check class distribution
class_counts = df['mapped_class'].value_counts()
print("Class distribution before undersampling:\n", class_counts)

# # Step 2: Find the smaller class size
# min_class_size = 3000

# # Step 3: Undersample the larger class
# df = (
#     df.groupby('mapped_class')  # Group by the class
#     .apply(lambda x: x.sample(n=min_class_size, random_state=42))  # Sample from each group
#     .reset_index(drop=True)  # Reset the index
# )

# # Step 4: Check class distribution after undersampling
# balanced_class_counts = df['mapped_class'].value_counts()
# print("Class distribution after undersampling:\n", balanced_class_counts)


In [ ]:
# df['language'].apply(len).sum()

In [ ]:
# len(set(item for sublist in df['language'] for item in sublist))

In [ ]:
# df.city.nunique()

In [28]:
# First, split into train + eval and test
X = df['tweets_concatenated'].values.tolist()
y = df['mapped_class'].values.tolist()
# First, split into train + eval and test
X_train_eval, X_test, y_train_eval, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Then, split train + eval into train and eval
X_train, X_eval, y_train, y_eval= train_test_split(X_train_eval, y_train_eval, test_size=0.2, random_state=0)  # 0.25 * 0.8 = 0.2

print(len(X_train))
print(len(X_eval))
print(len(X_test))


In [ ]:
# # Convert text to numerical features
# vectorizer = TfidfVectorizer()
# X_train_tfidf = vectorizer.fit_transform(X_train)
# X_test_tfidf = vectorizer.transform(X_test)

# # Train Naive Bayes
# model = MultinomialNB()
# model.fit(X_train_tfidf, y_train)

# # Predict and evaluate
# y_pred = model.predict(X_test_tfidf)
# print(classification_report(y_test, y_pred))

In [ ]:
CHECK_POINT = "google-bert/bert-base-multilingual-cased"
CACHE_DIR = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/.cache/huggingface/hub"  # Specify your custom directory

# Load tokenizer and model with the specified cache directory
tokenizer = AutoTokenizer.from_pretrained(CHECK_POINT, cache_dir=CACHE_DIR)
# model = BertForSequenceClassification.from_pretrained(CHECK_POINT, cache_dir=CACHE_DIR, num_labels=2)

In [ ]:
def encoding_and_labels(X, y):
    sentences = X
    label_str = y
    labels = [0 if label == "residential" else 1 for label in label_str]
    encodings = tokenizer(sentences, truncation=True, padding=True, return_tensors="pt") # , is_split_into_words=True
    return encodings, labels

train_encodings, train_labels = encoding_and_labels(X_train, y_train)
eval_encodings, eval_labels = encoding_and_labels(X_eval, y_eval)
test_encodings, test_labels = encoding_and_labels(X_test, y_test)

In [ ]:
class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = TwitterDataset(train_encodings, train_labels)
eval_dataset = TwitterDataset(eval_encodings, eval_labels)
test_dataset = TwitterDataset(test_encodings, test_labels)


In [31]:
test_dataset.labels

In [ ]:
def compute_metrics(eval_prediction):
    predictions = eval_prediction.predictions.argmax(-1)
    labels = eval_prediction.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    metrics = {'accuracy': acc,
               'precision': precision,
               'recall': recall,
               'f1': f1}
    return metrics

In [ ]:
OUTPUT_DIR = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/output_real"
LOGGING_DIR ="/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/output_real"
NUM_EPOCHS = 5
TRAIN_BATCH_SIZE = 32 #8
EVAL_BATCH_SIZE = 32 #32
WARMUP_RATIO = 0.01 # 200
WEIGHT_DECAY = 0.01
LOGGING_STEPS = 100 # 1000
LEARNING_RATE = 5e-6

# Calculate total training steps
total_train_steps = (len(train_labels) // TRAIN_BATCH_SIZE) * NUM_EPOCHS
print(total_train_steps)

# Calculate the saving steps to be aligned with logging steps
SAVING_STEPS = LOGGING_STEPS

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    warmup_ratio= WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    logging_dir=LOGGING_DIR,
    evaluation_strategy="steps",
    logging_steps=LOGGING_STEPS,
    save_steps=SAVING_STEPS,  # Set saving steps to align with logging steps
    learning_rate=LEARNING_RATE,
    metric_for_best_model="f1"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  #{'train':train_dataset,'eval':eval_dataset},
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
def compute_metrics(eval_prediction):
    predictions = eval_prediction.predictions.argmax(-1)
    labels = eval_prediction.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    metrics = {'accuracy': acc,
               'precision': precision,
               'recall': recall,
               'f1': f1}
    
    # convert label_id to label name
    mapping = {0: "residential", 1: "commercial"} 
    labels = [mapping[label_id] for label_id in labels]
    predictions = [mapping[prediction] for prediction in predictions]
    
    # create classification report
    report = classification_report(labels, predictions, zero_division=0)
    print("Classification Report:\n\n\n", report)
    
    return metrics

In [11]:
# model_trained = "/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/output_real/checkpoint-600"  # Specify your custom directory

# model = BertForSequenceClassification.from_pretrained(model_trained, cache_dir=CACHE_DIR, num_labels=2)

In [12]:
import warnings
warnings.filterwarnings('ignore')

test_args = TrainingArguments(
    output_dir='/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/output_synthetic',
    do_train=False,
    do_predict=True,
    per_device_eval_batch_size=32,   
    dataloader_drop_last=False,
)


trainer = Trainer(
    model=model,
    args=test_args,
    compute_metrics=compute_metrics,
) 

test_results = trainer.predict(test_dataset)

In [32]:
# # Extract predicted class indices
# y_pred = np.argmax(test_results.predictions, axis=1)
# # Extract true labels from test_dataset (assuming test_dataset has labels)
# y_test = test_dataset.labels # Adjust if labels are stored differently

# # Find indices where predictions do not match actual labels
# misclassified_indices = np.where(y_test != y_pred)[0]

# # Create a DataFrame of misclassified samples
# misclassified_samples = pd.DataFrame({
#     'Text': [X_test[i] for i in misclassified_indices],
#     'Actual Label': [y_test[i] for i in misclassified_indices],
#     'Predicted Label': [y_pred[i] for i in misclassified_indices]
# })
# misclassified_samples